####1️⃣ Prep: Create folders & split your Person.json into smaller files
This splits your uploaded /FileStore/tables/Person.json into individual JSON files so we can “feed” them into the stream one at a time.

In [0]:
import json, os, shutil, glob

# Paths
uploaded = "/dbfs/FileStore/tables/Person.json"       # already uploaded file
parts_dir = "/dbfs/FileStore/streaming_parts/"        # staging: all split files
input_dir = "/dbfs/FileStore/streaming_input/person/" # folder watched by stream

# Clean old runs
shutil.rmtree(parts_dir, ignore_errors=True)
shutil.rmtree(input_dir, ignore_errors=True)
os.makedirs(parts_dir, exist_ok=True)
os.makedirs(input_dir, exist_ok=True)

# Read uploaded file (handle both array JSON & NDJSON)
raw = open(uploaded, "r", encoding="utf-8").read().strip()
items = []
try:
    parsed = json.loads(raw)
    if isinstance(parsed, list):
        items = parsed
    else:
        items = [parsed]
except Exception:
    for line in raw.splitlines():
        line = line.strip()
        if line:
            try:
                items.append(json.loads(line))
            except:
                pass

# Write each record to its own file in staging folder
for i, obj in enumerate(items, start=1):
    fname = os.path.join(parts_dir, f"person_part_{i:03d}.json")
    with open(fname, "w", encoding="utf-8") as f:
        json.dump(obj, f)

print(f"✅ Created {len(items)} split files in: {parts_dir}")
print("Next: We will 'release' these into the input folder while stream runs.")

✅ Created 5 split files in: /dbfs/FileStore/streaming_parts/
Next: We will 'release' these into the input folder while stream runs.


####2️⃣ Helper: Release the next file into the stream input
Run this cell each time you want to simulate a new event arriving.

In [0]:
import shutil, glob, os

parts = sorted(glob.glob(parts_dir + "person_part_*.json"))
if not parts:
    print("❌ No more files to release.")
else:
    src = parts[0]
    dest = os.path.join(input_dir, os.path.basename(src))
    shutil.move(src, dest)
    print(f"📤 Released {os.path.basename(src)} to stream input folder.")

❌ No more files to release.


####3️⃣ Define the schema for the Person records

In [0]:
from pyspark.sql.types import StructType, IntegerType, StringType

person_schema = (StructType()
    .add("id", IntegerType())
    .add("firstname", StringType())
    .add("middlename", StringType())
    .add("lastname", StringType())
    .add("dob_year", IntegerType())
    .add("dob_month", StringType())  # month might be string or int; adjust if needed
    .add("gender", StringType())
    .add("salary", IntegerType())
)

####4️⃣ Create the streaming DataFrame

In [0]:
input_path = "/FileStore/streaming_input/person/"

person_stream = (spark.readStream
                 .schema(person_schema)
                 .json(input_path))

person_stream.printSchema()

root
 |-- id: integer (nullable = true)
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- dob_year: integer (nullable = true)
 |-- dob_month: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)



####5️⃣ Start console output stream
Keep this running in one cell — then switch back to cell 2 to “release” files. You should see each record appear as you release it.

In [0]:
query_console = (person_stream.writeStream
                 .format("console")
                 .outputMode("append")
                 .option("truncate", False)
                 .start())

####6️⃣ (Optional) Persist to Delta
If you also want to save processed records to Delta for later queries:

In [0]:
target_path = "/dbfs/tmp/ss_tutorial/target_delta/"
checkpoint_path = "/dbfs/tmp/ss_tutorial/checkpoint_delta/"

shutil.rmtree(target_path, ignore_errors=True)
shutil.rmtree(checkpoint_path, ignore_errors=True)

delta_query = (person_stream.writeStream
               .format("delta")
               .option("path", target_path)
               .option("checkpointLocation", checkpoint_path)
               .outputMode("append")
               .start())

Later, read saved data:


In [0]:
spark.read.format("delta").load(target_path).show()

+---+---------+----------+--------+--------+---------+------+------+
| id|firstname|middlename|lastname|dob_year|dob_month|gender|salary|
+---+---------+----------+--------+--------+---------+------+------+
|  3|  Robert |          |Williams|    2010|        3|     M|  4000|
|  4|   Maria |      Anne|   Jones|    2005|        5|     F|  4000|
|  2| Michael |      Rose|        |    2010|        3|     M|  4000|
|  1|   James |          |   Smith|    2018|        1|     M|  3000|
|  5|      Jen|      Mary|   Brown|    2010|        7|      |    -1|
+---+---------+----------+--------+--------+---------+------+------+



####7️⃣ Stop the streams when done

In [0]:
for q in spark.streams.active:
    q.stop()
print("✅ All active streams stopped.")

✅ All active streams stopped.


####🔹 How to use it in real-time:

- Run cell 1 — split your big JSON file into small chunks.

- Run cell 4 — start the stream by creating the DataFrame.

- Run cell 5 — start the console output.

- Run cell 2 repeatedly — each time you run it, one more JSON file is dropped into the input folder, and you’ll see a new record appear in the console output in real-time.

- When done, run cell 7 to stop the stream.

____
